# Ground Truth generation

# table of content
1) [Load stats](#load-stats)
2) [Show histograms and barplots](#show-histograms-and-barplots)
3) [Pie on heights](#pie-on-heights)

### Dependencies and general utils

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import open3d as o3d
import laspy
import pdal
import json
from tqdm import tqdm

### Generation

In [93]:
# Loading sources
src_folder_instances = r"..\data\split_testing\gt\final_gt\color_grp_full_tile_128_final\instances"
src_original_prediction = r"..\data\split_testing\gt\predictions\color_grp_full_tile_128_out.laz"
src_folder_result = r"..\data\split_testing\gt\final_gt"

In [94]:
# Load original and reset/create gt columns
full_tile = laspy.read(src_original_prediction)
full_tile.remove_extra_dim('gt_semantic_segmentation')
full_tile.add_extra_dim(laspy.ExtraBytesParams('gt_semantic_segmentation',type="uint16"))
full_tile.add_extra_dim(laspy.ExtraBytesParams('gt_instance_segmentation',type="uint16"))
print(full_tile.x)

<ScaledArrayView([2571634.37 2571634.37 2571634.37 ... 2571734.26 2571734.26 2571734.26])>


In [95]:
# Loop on gt instances and set the correct values in the full tile
list_instances_src = [x for x in os.listdir(src_folder_instances) if x.endswith('.laz')]
rounding = 2
semantic_layer = np.zeros(len(full_tile))
instance_layer = np.zeros(len(full_tile))
for id_instance, instance_src in tqdm(enumerate(list_instances_src), total=len(list_instances_src)):
    instance = laspy.read(os.path.join(src_folder_instances, instance_src))
    coords = list(zip(np.round(instance.x, rounding), np.round(instance.y, rounding), np.round(instance.z, rounding)))
    mask = np.array([(x,y,z) in coords for x, y, z in zip(np.round(full_tile.x, rounding), np.round(full_tile.y, rounding), np.round(full_tile.z, rounding))])
    semantic_layer[mask] = 1
    instance_layer[mask] = id_instance + 1

    assert np.sum(mask) == len(coords)

setattr(full_tile, 'gt_semantic_segmentation', semantic_layer)
setattr(full_tile, 'gt_instance_segmentation', instance_layer)

# save file
new_file = os.path.join(os.path.join(src_folder_result), os.path.basename(src_original_prediction).split('.laz')[0] + '_gt.laz')
full_tile.write(new_file)

100%|██████████| 33/33 [02:10<00:00,  3.94s/it]


In [96]:
# verify results and save file
print("Semantic results:")
for cat in set(full_tile.gt_semantic_segmentation):
    print(f"\tVal {cat}: number of points = {len(full_tile.gt_semantic_segmentation[full_tile.gt_semantic_segmentation == cat])}")
print("Instances results:")
for cat in set(full_tile.gt_instance_segmentation):
    print(f"\tVal {cat}: number of points = {len(full_tile.gt_instance_segmentation[full_tile.gt_instance_segmentation == cat])}")

Semantic results:
	Val 0: number of points = 377417
	Val 1: number of points = 17472
Instances results:
	Val 0: number of points = 377417
	Val 1: number of points = 480
	Val 2: number of points = 238
	Val 3: number of points = 505
	Val 4: number of points = 427
	Val 5: number of points = 337
	Val 6: number of points = 330
	Val 7: number of points = 475
	Val 8: number of points = 105
	Val 9: number of points = 709
	Val 10: number of points = 1066
	Val 11: number of points = 178
	Val 12: number of points = 127
	Val 13: number of points = 1358
	Val 14: number of points = 154
	Val 15: number of points = 116
	Val 16: number of points = 546
	Val 17: number of points = 713
	Val 18: number of points = 425
	Val 19: number of points = 973
	Val 20: number of points = 564
	Val 21: number of points = 480
	Val 22: number of points = 661
	Val 23: number of points = 113
	Val 24: number of points = 416
	Val 25: number of points = 405
	Val 26: number of points = 757
	Val 27: number of points = 776
	Val 

In [11]:

for col in list(full_tile.point_format.dimension_names):
    print(col)

X
Y
Z
intensity
return_number
number_of_returns
synthetic
key_point
withheld
overlap
scanner_channel
scan_direction_flag
edge_of_flight_line
classification
user_data
scan_angle
point_source_id
gps_time
red
green
blue
PredSemantic
gt_semantic_segmentation
PredInstance
gt_instance_segmentation
